## EDA PROCESS FROM CSV

Luis Angel Garcia (2230177)


Libraries

In [58]:
import pandas as pd
import logging
import matplotlib.pyplot as plt
import seaborn as sns


import sys

sys.path.append("../config_documents")

import postgres_setup as pg
import importlib
importlib.reload(pg)

<module 'postgres_setup' from '/mnt/d/Universidad/ETL/workshop2_spotify/notebook/../config_documents/postgres_setup.py'>

Loggin config

In [ ]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

Postgress config

In [ ]:
pg.crear_usuario_y_bd()

conn = pg.conectar_etl()

pg.crear_tabla_spotify_data(conn)
pg.crear_tabla_grammy_awards(conn)

Load csv

In [ ]:
df = pd.read_csv("../datasets/spotify_dataset.csv")
logging.info("Dataset cargado correctamente.")

General Analisys

In [ ]:
logging.info(f"Dimensiones del dataset: {df.shape}")
logging.info(f"Columnas: {df.columns.tolist()}")
logging.info("Valores nulos por columna:")
logging.info(df.isnull().sum())
logging.info("Tipos de datos:")
logging.info(df.dtypes)
logging.info("Estadísticas descriptivas:")
logging.info(df.describe(include='all').transpose())

Cleaning

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)
logging.info("Columna 'Unnamed: 0' eliminada.")

df.drop(columns=['track_id'], inplace=True)
logging.info("Columna 'track_id' eliminada.")

missing_before = df.isnull().sum().sum()
df.dropna(inplace=True)
missing_after = df.isnull().sum().sum()
logging.info(f"Se eliminaron {missing_before - missing_after} valores nulos.")

df = df[df['duration_ms'] < 900000] 
df = df[df['loudness'] > -40]        
logging.info("Outliers eliminados en 'duration_ms' y 'loudness'.")

df.reset_index(drop=True, inplace=True)
logging.info("Índice reseteado.")

Graphic analysis

Load to csv

In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
pg.insertar_datos_spotify_data(conn, df)

In [ ]:
df['duration_min'] = df['duration_ms'] / 60000

features = ['popularity', 'duration_min', 'danceability', 'energy', 'valence', 'tempo']

for col in features:
    plt.figure(figsize=(8, 4))
    sns.histplot(df[col], bins=40, kde=True, color='skyblue')
    plt.axvline(df[col].mean(), color='red', linestyle='--', label=f"Media: {df[col].mean():.2f}")
    plt.axvline(df[col].median(), color='green', linestyle='--', label=f"Mediana: {df[col].median():.2f}")
    plt.title(f'Distribución de {col}')
    plt.xlabel(col)
    plt.ylabel('Frecuencia')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
df['duration_min'] = df['duration_ms'] / 60000

atributos = ['duration_min', 'tempo']

for col in atributos:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[col], color='lightblue')
    plt.title(f'Boxplot de {col}')
    plt.xlabel(col)
    plt.tight_layout()
    plt.show()


In [ ]:
Q1 = df['duration_min'].quantile(0.25)
Q3 = df['duration_min'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

canciones_outliers = df[
    (df['duration_min'] < limite_inferior) | 
    (df['duration_min'] > limite_superior)
]
print(limite_superior)
print(limite_inferior)
canciones_outliers[['track_name', 'artists', 'duration_min']]

6.5339
0.720766666666667


,track_name,artists,duration_min
253,Rain - Long Ver.,Motohiro Hata,7.455100
650,いきのこり●ぼくら,Ichiko Aoba,6.768383
752,Bad News Is Coming,Luther Allison,7.425550
851,All Too Well - Jake's Version,John Elliott,9.634400
896,Alone in the Dark Mansion of Madness,Harley Poe,6.731850
...,...,...,...
113740,Divine / Sailing / Time / Hymn - Acoustic,Phil Wickham,7.739967
113753,Hope Of The Ages - Live,Hillsong Worship;Reuben Morgan,6.610767
113767,Open Space,Housefires,7.616350
113777,Ain’t No Grave - Live,Bethel Music;Molly Skaggs,8.221550
